<a href="https://colab.research.google.com/github/santhimaddipudi/ML_Handson/blob/main/ML_Databricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

from pyspark.sql import SparkSession

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=ca0dea7e0d034efb16aaa6f5d6b948317a2ef566efc6db7ec74667e197fbf279
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
# Create a SparkSession
spark = SparkSession.builder.appName("ImputerExample").getOrCreate()

In [6]:
from pyspark.ml.feature import Imputer

df = spark.createDataFrame([
    (1.0, float("nan")),
    (2.0, float("nan")),
    (float("nan"), 3.0),
    (4.0, 4.0),
    (5.0, 5.0)
], ["a", "b"])
df.show()

+---+---+
|  a|  b|
+---+---+
|1.0|NaN|
|2.0|NaN|
|NaN|3.0|
|4.0|4.0|
|5.0|5.0|
+---+---+



In [8]:
imputer = Imputer(inputCols=["a", "b"], outputCols=["a", "b"])
model = imputer.fit(df)

model.transform(df).show()

+---+---+
|  a|  b|
+---+---+
|1.0|4.0|
|2.0|4.0|
|3.0|3.0|
|4.0|4.0|
|5.0|5.0|
+---+---+

